In [1]:
import ngmix
from ngmix.medsreaders import NGMixMEDS
import numpy as np
import os
import time
from astropy.table import Table, vstack, hstack


import numpy as np
import pandas as pd

import sklearn.neighbors as neighbors

from importlib import reload
import copy
import synthetic.render.generator as gen

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

import multiprocessing as mp

import sys
import os
import math
import logging
import time
import galsim
import fitsio as fio

import ngmix

import synthetic.render.render as render
import synthetic.render.icl as icl
import synthetic.render.frame as frame

import images
import meds
import subprocess
import psfex
from argparse import ArgumentParser


In [2]:

class MetacalFitter(object):
    def __init__(self, medsfile, seed=None):
        self.medsfile = medsfile

        # for some reason this is bugged if you
        self.meds = NGMixMEDS(medsfile)
        self.cat = self.meds.get_cat()
        self.Nobjs = len(self.cat)

        self.set_seed(seed)

        return

    def set_seed(self, seed=None):
        if seed is None:
            # current time in microseconds
            seed = int(1e6*time.time())

        self.seed = seed

        return

    def get_obs_list(self, iobj):
        return self.meds.get_obslist(iobj)

    def get_jacobians(self, iobj):
        Njac = len(self.meds.get_jacobian_list(iobj))

        jacobians = [self.meds.get_ngmix_jacobian(iobj, icutout)
                     for icutout in range(Njac)]

        return jacobians

    def get_obj_info(self, iobj):
        '''
        Setup object property dictionary used to compile fit params later on
        '''

        obj = self.meds[iobj]

        # Mcal object properties
        obj_info = {}

        obj_info['meds_indx'] = iobj
        obj_info['id'] = obj['id']
        obj_info['ra'] = obj['ra']
        obj_info['dec'] = obj['dec']
        obj_info['X_IMAGE'] = obj['X_IMAGE']
        obj_info['Y_IMAGE'] = obj['Y_IMAGE']

        return obj_info

    def get_prior(self, pixel_scale):
        '''
        pix_scale: The pixel scale of the image, in arcsec / pixel
        '''

        # This bit is needed for ngmix v2.x.x
        # won't work for v1.x.x
        #rng = np.random.RandomState(self.seed)

        # prior on ellipticity.  The details don't matter, as long
        # as it regularizes the fit.  This one is from Bernstein & Armstrong 2014

        g_sigma = 0.3
        g_prior = ngmix.priors.GPriorBA(g_sigma)

        # 2-d gaussian prior on the center
        # row and column center (relative to the center of the jacobian, which would be zero)
        # and the sigma of the gaussians

        # units same as jacobian, probably arcsec
        row, col = 0.0, 0.0
        row_sigma, col_sigma = pixel_scale, pixel_scale # use pixel_scale as a guess
        cen_prior = ngmix.priors.CenPrior(row, col, row_sigma, col_sigma)

        # T prior.  This one is flat, but another uninformative you might
        # try is the two-sided error function (TwoSidedErf).
        # NOTE: T units are arcsec^2 but can be slightly negative, especially for
        # stars if the PSF is mis-estimated

        Tminval = -1.0 # arcsec squared
        Tmaxval = 1000
        T_prior = ngmix.priors.FlatPrior(Tminval, Tmaxval)

        # similar for flux.  Make sure the bounds make sense for
        # your images

        Fminval = -1.e1
        Fmaxval = 1.e5
        F_prior = ngmix.priors.FlatPrior(Fminval, Fmaxval)

        # now make a joint prior.  This one takes priors
        # for each parameter separately
        prior = ngmix.joint_prior.PriorSimpleSep(
            cen_prior,
            g_prior,
            T_prior,
            F_prior
            )

        return prior

    def add_mcal_responsivities(self, mcal_res, mcal_shear):
        '''
        Compute and add the mcal responsivity values to the output
        result dict from get_metacal_result()

        NOTE: These are only for the selection-independent component!
        '''

        # Define full responsivity matrix, take inner product with shear moments
        r11 = (mcal_res['1p']['g'][0] - mcal_res['1m']['g'][0]) / (2*mcal_shear)
        r12 = (mcal_res['2p']['g'][0] - mcal_res['2m']['g'][0]) / (2*mcal_shear)
        r21 = (mcal_res['1p']['g'][1] - mcal_res['1m']['g'][1]) / (2*mcal_shear)
        r22 = (mcal_res['2p']['g'][1] - mcal_res['2m']['g'][1]) / (2*mcal_shear)

        R = [ [r11, r12], [r21, r22] ]
        Rinv = np.linalg.inv(R)
        gMC = np.dot(Rinv,
                     mcal_res['noshear']['g']
                     )

        MC = {
            'r11':r11, 'r12':r12,
            'r21':r21, 'r22':r22,
            'g1_MC':gMC[0], 'g2_MC':gMC[1]
        }

        mcal_res['MC'] = MC

        return mcal_res

    def mcal_dict2tab(self, mcal, obj_info):
        '''
        mcal is the dict returned by ngmix.get_metacal_result()

        obj_info is an array with MEDS identification info like id, ra, dec
        not returned by the function
        '''

        # Annoying, but have to do this to make Table from scalars
        for key, val in obj_info.items():
            obj_info[key] = np.array([val])

        tab_names = ['noshear', '1p', '1m', '2p', '2m','MC']
        for name in tab_names:
            tab = mcal[name]

            for key, val in tab.items():
                tab[key] = np.array([val])

            mcal[name] = tab

        id_tab = Table(data=obj_info)

        tab_noshear = Table(mcal['noshear'])
        tab_1p = Table(mcal['1p'])
        tab_1m = Table(mcal['1m'])
        tab_2p = Table(mcal['2p'])
        tab_2m = Table(mcal['2m'])
        tab_MC = Table(mcal['MC'])

        join_tab = hstack([id_tab, hstack([tab_noshear,
                                           tab_1p,
                                           tab_1m,
                                           tab_2p,
                                           tab_2m,
                                           tab_MC
                                           ],
                                          table_names=tab_names)
                           ]
                          )

        return join_tab

    def fit_obj(self, iobj, pars=None, ntry=4, psf_model='gauss',
                gal_model='gauss', vb=False):
        '''
        Run metacal fit for a single object of given index

        pars: mcal running parameters
        '''

        obj_info = self.get_obj_info(iobj)

        # Fits need a list of ngmix.Observation objects
        obs_list = self.get_obs_list(iobj)

        # Get pixel scale from image jacobian
        jac_list = self.get_jacobians(iobj)
        pixel_scale = jac_list[0].get_scale()

        if pars is None:
            # standard mcal run parameters
            mcal_shear = 0.01
            lm_pars = {'maxfev':2000, 'xtol':5.0e-5, 'ftol':5.0e-5}
            max_pars = {'method':'lm', 'lm_pars':lm_pars, 'find_center':True}
            metacal_pars = {'step':mcal_shear}
        else:
            mcal_shear = metacal_pars['step']
            max_pars = pars['max_pars']
            metacal_pars = pars['metacal_pars']

        prior = self.get_prior(pixel_scale)

        Tguess = 4*pixel_scale**2

        # setup run bootstrapper
        mcb = ngmix.bootstrap.MaxMetacalBootstrapper(obs_list)

        # Run the actual metacalibration step on the observed source
        mcb.fit_metacal(psf_model, gal_model, max_pars, Tguess, prior=prior,
                        ntry=ntry, metacal_pars=metacal_pars)

        mcal_res = mcb.get_metacal_result() # this is a dict

        # Add selection-independent responsitivities
        mcal_res = self.add_mcal_responsivities(mcal_res, mcal_shear)

        if vb is True:
            r11 = mcal_res['MC']['r11']
            r22 = mcal_res['MC']['r22']
            print(f'i={iobj}: R11: {r11:.3}; R22: {r22:.3} ')

        mcal_tab = self.mcal_dict2tab(mcal_res, obj_info)

        return mcal_tab


In [8]:

medsfile = "testmeds.fits"
# medsfile = "gauss_uberseg_meds.fits.fz"
outfile = "metacal_res.fits"
outdir = None
start = None
end = 100
clobber = True
vb = True #args.vb # if True, prints out values of R11/R22 for every galaxy

if outdir is not None:
    outfile = os.path.join(outdir, outfile)

fitter = MetacalFitter(medsfile)

if start is None:
    start = 0
if end is None:
    end = fitter.Nobjs

# Can set mcal parameters here if you want something beyond the default
# in fit_obj()
pars = None

Tstart = time.time()

mcal_tab = []
for iobj in range(start, end):
    mcal_tab.append(
        fitter.fit_obj(iobj, pars=pars, vb=vb)
        )

mcal_tab = vstack(mcal_tab)

Tend = time.time()

T = Tend - Tstart
print(f'Total fitting and stacking time: {T} seconds')

if vb is True:
    print(f'Writing out mcal results to {outfile}...')
mcal_tab.write(outfile, overwrite=clobber)

if vb is True:
    print('Done!')



i=0: R11: 0.9; R22: 0.995 
i=1: R11: -0.137; R22: 0.817 
i=2: R11: 0.662; R22: 0.797 
i=3: R11: 2.9; R22: -0.607 
i=4: R11: 0.461; R22: 0.986 
i=5: R11: -0.52; R22: -0.604 
i=6: R11: 0.0528; R22: 0.378 
i=7: R11: 0.849; R22: 0.985 
i=8: R11: 8.26; R22: 49.7 
i=9: R11: 1.09; R22: 0.689 
i=10: R11: -0.743; R22: 0.659 
i=11: R11: 1.17; R22: -0.0527 
i=12: R11: 1.13; R22: 0.962 
i=13: R11: 0.361; R22: 0.473 
i=14: R11: 0.798; R22: 0.806 
i=15: R11: 1.36; R22: 0.211 
i=16: R11: 4.82; R22: -0.226 
i=17: R11: 0.744; R22: 0.877 
i=18: R11: 4.28; R22: -1.86 
i=19: R11: 1.02; R22: 0.95 
i=20: R11: 0.777; R22: 0.827 
i=21: R11: 0.994; R22: 0.786 
i=22: R11: 0.968; R22: 0.326 
i=23: R11: 0.971; R22: 1.01 
i=24: R11: -0.257; R22: 1.2 
i=25: R11: 0.343; R22: 1.31 
i=26: R11: 0.798; R22: 0.818 
i=27: R11: 0.633; R22: 1.06 
i=28: R11: 1.47; R22: 0.804 
i=29: R11: -0.24; R22: -0.0878 
i=30: R11: 2.08; R22: -3.25 
i=31: R11: 0.258; R22: 1.11 
i=32: R11: -1.49; R22: 0.573 
i=33: R11: 1.07; R22: 1.01 
i=3

In [5]:
res = fio.read("metacal_res.fits")

In [44]:
res["g2_MC"]

array([-0.18944926,  0.04469121, -0.19684356, -1.91172157, -0.00952851,
        0.41365582,  9.36571582, -0.2976869 , -0.05498025, -0.71816855])

In [45]:
res["g1_MC"]

array([ 0.20496091, -0.12132284,  0.29265646,  0.31355887, -0.05458493,
        0.41673458, -6.60819758,  0.31560374,  0.06995793,  0.48539343])

In [10]:
res.dtype

dtype([('meds_indx', '>i8'), ('id', '>i8'), ('ra', '>f8'), ('dec', '>f8'), ('X_IMAGE', '>f8'), ('Y_IMAGE', '>f8'), ('flags_noshear', '>i8'), ('nfev_noshear', '>i8'), ('ier_noshear', '>i8'), ('errmsg_noshear', '<U169'), ('pars_noshear', '>f8', (6,)), ('pars_err_noshear', '>f8', (6,)), ('pars_cov0_noshear', '>f8', (6, 6)), ('pars_cov_noshear', '>f8', (6, 6)), ('model_noshear', '<U5'), ('g_noshear', '>f8', (2,)), ('g_cov_noshear', '>f8', (2, 2)), ('flux_noshear', '>f8'), ('flux_err_noshear', '>f8'), ('T_noshear', '>f8'), ('T_err_noshear', '>f8'), ('lnprob_noshear', '>f8'), ('s2n_numer_noshear', '>f8'), ('s2n_denom_noshear', '>f8'), ('npix_noshear', '>i8'), ('chi2per_noshear', '>f8'), ('dof_noshear', '>i8'), ('s2n_w_noshear', '>f8'), ('s2n_noshear', '>f8'), ('ntry_noshear', '>i8'), ('s2n_r_noshear', '>f8'), ('T_r_noshear', '>f8'), ('psf_T_r_noshear', '>f8'), ('gpsf_noshear', '>f8', (2,)), ('Tpsf_noshear', '>f8'), ('flags_1p', '>i8'), ('nfev_1p', '>i8'), ('ier_1p', '>i8'), ('errmsg_1p', '<U

In [6]:
res[""]

array([(0,  1, 89.99926136, -0.16698827, 2510.56958008, 223.40190125, 0,  43, 1, 'Both actual and predicted relative reductions in the sum of squares\n  are at most 0.000050', [-2.64854087e-01, -2.66330449e-01,  1.81492494e-01, -1.92302730e-01,  1.80981878e-01,  1.17761339e+02], [6.22006183e-03, 7.06164536e-03, 2.89065907e-02, 2.91010033e-02, 9.54823155e-03, 1.57727425e+00], [[ 6.59702137e-06, -9.78710184e-07,  3.09296372e-08, -6.72302223e-08,  2.42768999e-08,  2.50484359e-06], [-9.78710184e-07,  8.50296310e-06, -4.19133549e-08,  8.25424462e-08, -4.63135001e-08, -6.02669176e-06], [ 3.09296372e-08, -4.19133549e-08,  1.42479451e-04, -1.66961212e-05, -5.10440830e-06, -1.27837617e-03], [-6.72302223e-08,  8.25424462e-08, -1.66961212e-05,  1.44402400e-04,  5.60269049e-06,  1.37980690e-03], [ 2.42768999e-08, -4.63135001e-08, -5.10440830e-06,  5.60269049e-06,  1.55454884e-05,  1.78099202e-03], [ 2.50484359e-06, -6.02669176e-06, -1.27837617e-03,  1.37980690e-03,  1.78099202e-03,  4.24202197e-01

In [ ]:
parser = ArgumentParser()

parser.add_argument('medsfile', type=str,
                    help='MEDS file to process')
parser.add_argument('outfile', type=str,
                    help='Output filename')
parser.add_argument('-outdir', type=str, default=None,
                    help='Output directory')
parser.add_argument('-start', type=int, default=None,
                    help='Starting index for MEDS processing')
parser.add_argument('-end', type=int, default=None,
                    help='Ending index for MEDS processing')
parser.add_argument('--clobber', action='store_true', default=False,
                    help='Set to overwrite output files')
parser.add_argument('--vb', action='store_true', default=False,
                    help='Make verbose')

In [ ]:
#     args = parser.parse_args()
main(args)